In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D
!pip install -U git+https://github.com/twopirllc/pandas-ta
import pandas_ta as pta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 6.4 MB 10.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [2]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)

    if len(data.columns) == 7:
      data = data.iloc[:, 1:]

    data = np.array(data)
    data = data.astype(float)
    max_index = data.shape[0]-which_future_or_past
    for i in range(each_row_past,max_index):

        rows = data[i-each_row_past:i, :]
        past_candles = []
        for z in range(1, how_many_past_candles+1):
          past_candles.append((data[i-z][3]+data[i-z][0])/2)
        past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append((data[i+z][3]+data[i+z][0])/2)
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > past_candles:
          sugg = 1
        else:
          sugg = 0

        df = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","Adj Close","Volume"])

        df.index.name = "Date"

        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{sugg}.png"
        

        fig, _a = mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                            returnfig=True, tight_layout=True,figsize =(1.5,1.5))
        
        fig.savefig(address)
        fig.clear()
        plt.close(fig)

        del fig,_a
        if i % 10 == 0:
            print(f"{i}/{max_index}", end=" ")
        if i % 300 == 0:
          print(" ")
       

    print("")


def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd","ltc-usd","xrp-usd","bnb-usd"]
download_data(symbols,"max","1d")

Symbols : 1500
 


In [4]:
how_many_future_candles = 20
how_many_past_candles = 1
each_row_past = 150

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 6
Processing File:
File Number 1: 150/1698 160/1698 170/1698 180/1698 190/1698 200/1698 210/1698 220/1698 230/1698 240/1698 250/1698 260/1698 270/1698 280/1698 290/1698 300/1698  
310/1698 320/1698 330/1698 340/1698 350/1698 360/1698 370/1698 380/1698 390/1698 400/1698 410/1698 420/1698 430/1698 440/1698 450/1698 460/1698 470/1698 480/1698 490/1698 500/1698 510/1698 520/1698 530/1698 540/1698 550/1698 560/1698 570/1698 580/1698 590/1698 600/1698  
610/1698 620/1698 630/1698 640/1698 650/1698 660/1698 670/1698 680/1698 690/1698 700/1698 710/1698 720/1698 730/1698 740/1698 750/1698 760/1698 770/1698 780/1698 790/1698 800/1698 810/1698 820/1698 830/1698 840/1698 850/1698 860/1698 870/1698 880/1698 890/1698 900/1698  
910/1698 920/1698 930/1698 940/1698 950/1698 960/1698 970/1698 980/1698 990/1698 1000/1698 1010/1698 1020/1698 1030/1698 1040/1698 1050/1698 1060/1698 1070/1698 1080/1698 1090/1698 1100/1698 1110/1698 1120/1698 1130/1698 1140/1698 1150/1698 1160/1698 1170/1698

11586

In [6]:
shutil.make_archive(folder_name,"zip",f"/content/extracted/{folder_name}/")
#shutil.unpack_archive("/content/extracted/084505.zip","/content/extracted/455")

'/content/191751.zip'

In [5]:
#folder_name = '112904'

label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name,0)

    dim = (128, 128)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255
print(data.shape)
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.2,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

(11586, 128, 128)
xTrain : 9268 \ xTest : 2318
yn: 6178 nn: 5408


In [3]:
model = Sequential()

model.add(Conv2D(128,    (3, 3),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],1)))
model.add(Conv2D(100,    (3, 3),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,    (3, 3),activation="relu",))
model.add(Conv2D(64,    (3, 3),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,    (2, 2),activation="relu",))
model.add(Conv2D(32,    (2, 2),activation="relu",))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

NameError: ignored

In [ ]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#model.fit(xTrain,yTrain,batch_size=64,epochs=30,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)
model.fit(xTrain,yTrain,batch_size=32,epochs=20,validation_data=(xTest,yTest))


Epoch 1/20
  2/290 [..............................] - ETA: 47:52 - loss: 2.2425 - accuracy: 0.5469